In [ ]:
using PyPlot

In [ ]:
"""plot line y = w*x+b"""
function plotline(w,b;
                  xmin=-100,xmax=100)
    xsamples = [xmin, xmax]
    plot(xsamples, [w*x+b for x in xsamples], color="black")
end

In [ ]:
"""plot function y=f(x)"""
function plotfunc(f;
                  xmin=-100,xmax=100,nsamples=100)
    xsamples = linspace(xmin,xmax,nsamples)
    plot(xsamples, [f(x) for x in xsamples], color="black")
end

In [ ]:
# Generate and plot data
srand(2)
n = 20
function sample_data(num_points)
    x = rand(num_points)*10
    y = .2 + .2*x + .1*sin(x) + .03*randn(num_points) - .1*(x/6).^2
    return x,y
end
x,y = sample_data(n)

"""function to plot the above data"""
function plotdata(x=x,y=y; margin=.05)
    hold(true)
    plot(x,y,"o")
    xlabel("x")
    ylabel("y")
    range_y = maximum(y) - minimum(y)
    range_x = maximum(x) - minimum(x)
    ylim([minimum(y)-margin*range_y,maximum(y)+margin*range_y])
    xlim([minimum(x)-margin*range_x,maximum(x)+margin*range_x])
end
plotdata()

# Approximating with the mean

In [ ]:
# the mean solves a very simple least squares problem:
X = ones(n,1)
w = X\y

In [ ]:
# check the solution to our least squares problem is the mean
abs(mean(y) - w[1])

In [ ]:
# plot the fit
clf() # clears the previous figure
plotdata()
plotline(0, w[1])

# Approximating with a line

In [ ]:
X = [copy(x) ones(length(x))]

In [ ]:
w = X\y

In [ ]:
# plot the fit
clf()
plotdata()
plotline(w[1], w[2])

In [ ]:
# plot fit on out of sample data
clf()
plotdata()
plotline(w[1], w[2])

xtest,ytest = sample_data(20)
plotdata(xtest,ytest)

# Approximating with a polynomial

In [ ]:
# first, construct a Vandermonde matrix
max_order = 10

X = zeros(n, max_order+1)
for k=0:max_order
    X[:,k+1] = x.^k
end
X

In [ ]:
# solve least squares problem
w = X\y

In [ ]:
"""computes our polynomial fit evaluated at x"""
function p(x; order = max_order, w = w)
    y = 0
    for k=0:order
        y += w[k+1]*x^k
    end
    return y
end

In [ ]:
# plot fit
clf()
plotfunc(x -> p(x, order=max_order, w=w), xmin=0, xmax=9)
plotdata()

In [ ]:
# plot fit on out of sample data
clf()
plotfunc(x -> p(x, order=max_order, w=w), xmin=0, xmax=9)
plotdata()

xtest,ytest = sample_data(20)
plotdata(xtest,ytest)

# Choosing the best model order

In [ ]:
max_model_order = 10
rmse = Array(Float64, max_model_order+1) # array to store root mean square model errors
xtest,ytest = sample_data(50) # generate test set

for model_order = 0:max_model_order
    # form Vandermonde matrix
    X = zeros(n, model_order+1)
    for k=0:model_order
        X[:,k+1] = x.^k
    end
    
    # solve least squares problem
    w = X\y
    
    # compute test error
    ptest = [p(x, order=model_order, w=w) for x in xtest]
    rmse[model_order+1] = sqrt(mean((ytest - ptest).^2))
end
rmse

In [ ]:
xlabel("model order")
ylabel("rmse")
plot(rmse)

# Bootstrap estimators

In [ ]:
# sample K data sets of n samples each and compute a model on each
# see how the models vary
n = 20
K = 10

models = zeros(K,2)
for k=1:K
    xk,yk = sample_data(n)
    Xk = [xk ones(n)]
    wk = Xk \ yk
    models[k,:] = wk
end

In [ ]:
# histogram of the distribution of the first coefficient
# could use to compute, eg, confidence intervals

plt[:hist](models[:,1])

In [ ]:
mean(models,1)

In [ ]:
var(models,1)

In [ ]:
# can sample with replacement using rand
rand(1:15, 5)

# eg,
a = 0:.1:1.5
s = rand(1:15, 5)
[s a[s]]

In [ ]:
# resample K bootstrap data sets of n samples each and compute a model on each
# see how the models vary
n = 20
K = 100

x,y = sample_data(n)

models = zeros(K,2)
for k=1:K
    mysample = sample(1:n,n)
    xk,yk = x[mysample], y[mysample]
    Xk = [xk ones(n)]
    wk = Xk \ yk
    models[k,:] = wk
end

In [ ]:
plt[:hist](models[:,1])

In [ ]:
mean(models,1)

In [ ]:
# as K increases, mean of the bootstrap models should converge to 
# the model fit on the original data set
X = [x ones(n)]
w = X \ y

In [ ]:
var(models,1)